In [9]:
from google.cloud import bigquery;
from google.cloud.bigquery import dbapi;
import pandas as pd;

In [10]:
client = bigquery.Client("mining-clinical-decisions");

/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [11]:
conn = dbapi.connect(client);

## Test Query to make sure things are working

In [12]:
query = "SELECT * FROM `mining-clinical-decisions.starr_datalake2018.adt` LIMIT 10"; 
resultsDF = pd.read_sql_query(query, conn)

In [13]:
resultsDF

,jc_uid,pat_enc_csn_id_coded,event_id_coded,event_type_c,event_type,event_subtype_c,event_subtype,department_id,pat_class_c,pat_class,...,seq_num_in_enc,seq_num_in_bed_min,labor_status_c,first_ip_in_ip_yn,loa_reason_c,data_source,effective_time_jittered_utc,effective_time_jittered,event_time_jittered_utc,event_time_jittered
0,JCe4b0f8,131248255400,26794006,8,Leave of Absence Out,1,Original,120301002,172,SNF,...,43,1,None,None,4.0,CLARITY_SHC,2018-05-08 16:54:00+00:00,2018-05-08 09:54:00,2018-05-08 16:55:00+00:00,2018-05-08 09:55:00
1,JCd33188,131251163697,27554470,9,Leave of Absence Return,1,Original,120301002,172,SNF,...,32,1,None,None,NaN,CLARITY_SHC,2018-05-12 22:15:00+00:00,2018-05-12 15:15:00,2018-05-12 22:39:00+00:00,2018-05-12 15:39:00
2,JCeadc76,131249233204,26700504,9,Leave of Absence Return,1,Original,120301002,172,SNF,...,10,1,None,None,NaN,CLARITY_SHC,2018-03-15 15:30:00+00:00,2018-03-15 08:30:00,2018-03-15 16:04:00+00:00,2018-03-15 09:04:00
3,JCe00dd9,131247873288,26274432,8,Leave of Absence Out,1,Original,120301002,172,SNF,...,29,1,None,None,4.0,CLARITY_SHC,2018-02-11 17:04:00+00:00,2018-02-11 09:04:00,2018-02-11 17:06:00+00:00,2018-02-11 09:06:00
4,JCe5d92e,131250281284,27000067,9,Leave of Absence Return,1,Original,120301002,172,SNF,...,4,1,None,None,NaN,CLARITY_SHC,2018-05-17 01:24:00+00:00,2018-05-16 18:24:00,2018-05-17 01:24:00+00:00,2018-05-16 18:24:00
5,JCeadc76,131249233204,26699522,8,Leave of Absence Out,1,Original,120301002,172,SNF,...,9,1,None,None,4.0,CLARITY_SHC,2018-03-15 14:18:00+00:00,2018-03-15 07:18:00,2018-03-15 14:18:00+00:00,2018-03-15 07:18:00
6,JCe0a8cc,131250719981,27264341,9,Leave of Absence Return,1,Original,120301002,172,SNF,...,15,1,None,None,NaN,CLARITY_SHC,2018-06-07 23:10:00+00:00,2018-06-07 16:10:00,2018-06-07 23:32:00+00:00,2018-06-07 16:32:00
7,JCceb968,131257678868,29156948,8,Leave of Absence Out,1,Original,120301002,172,SNF,...,14,1,None,None,4.0,CLARITY_SHC,2018-10-28 12:15:00+00:00,2018-10-28 05:15:00,2018-10-28 15:21:00+00:00,2018-10-28 08:21:00
8,JCd1e561,131256499078,28900775,9,Leave of Absence Return,1,Original,120301002,172,SNF,...,17,1,None,None,NaN,CLARITY_SHC,2018-10-15 10:00:00+00:00,2018-10-15 03:00:00,2018-10-15 10:05:00+00:00,2018-10-15 03:05:00
9,JCdd60ca,131249935974,27076627,9,Leave of Absence Return,1,Original,120301002,172,SNF,...,26,1,None,None,NaN,CLARITY_SHC,2018-05-23 21:14:00+00:00,2018-05-23 14:14:00,2018-05-23 21:14:00+00:00,2018-05-23 14:14:00


## TDD  by day, by patient

-- TDD by day by patient. Insulin not pumps or U500 or rare. Patient without AKI during that encounter. 

In [12]:
query = open('Inpatient_info.sql', 'r')

In [13]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [14]:
tdd_subqueries_data.head(1)

,jc_uid,pat_enc_csn_id_coded,units,mar_action,med_description,route,medication_id,taken_time_jittered
0,JCcc0c77,131257485179,1,Given,INSULIN LISPRO 100 UNIT/ML SC MODERATE SCALE F...,Subcutaneous,541453,2018-09-19 11:44:00


In [15]:
tdd_subqueries_data.to_csv("./Data/InsulinDosing.csv")

In [16]:
query.close()

## TDD where BG was from 100-180

-- TDD on days where BG was 100-180 and at least 3 glucose readings in range

In [14]:
query = open('patient_date_numBG_minBG_maxBG_TDD.sql', 'r')

In [15]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [16]:
tdd_subqueries_data.head(1)

,jc_uid,patientDate,pat_enc_csn_id_coded,numBG,minBG,maxBG,TDD
0,JCd5154f,2013-05-22,131024269168,4,158.0,169.0,5.0


In [17]:
tdd_subqueries_data.to_csv("./Data/patient_date_numBG_minBG_maxBG_TDD.csv")

In [18]:
query.close()

## Age and Sex

DOB and Sex Query

In [25]:
query = open('age_and_sex.sql', 'r')

In [26]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [27]:
tdd_subqueries_data.head(1)

,rit_uid,gender,dob
0,JCcb6582,Female,1930-03-22


In [28]:
tdd_subqueries_data.to_csv("./Data/age_and_sex.csv")

In [29]:
query.close()

## A1C

Query for a1c

In [43]:
query = open('a1c.sql', 'r')

In [44]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [45]:
tdd_subqueries_data.head(10)

,rit_uid,pat_enc_csn_id_coded,lab_name,taken_time_jittered,ord_value,reference_unit,result_in_range_yn,result
0,JCcb657e,6648732,Hemoglobin A1c,2004-04-15 09:25:00,5.4,%,None,NaN
1,JCcb657e,6648732,Hemoglobin A1c,2005-09-20 10:11:00,5.3,%,None,NaN
2,JCcb657e,131008252292,Hemoglobin A1c,2010-01-11 10:13:00,5.6,%,None,NaN
3,JCcb65a9,4764750,Hemoglobin A1c,2004-07-15 05:10:00,7.0,%,None,NaN
4,JCcb65d7,14536711,Hemoglobin A1c,2000-08-26 12:40:00,6.9,%,None,NaN
5,JCcb65d7,14536711,Hemoglobin A1c,2001-07-21 14:05:00,7.4,%,None,NaN
6,JCcb65e1,6641909,Hemoglobin A1c,2000-07-03 16:00:00,5.7,%,Y,NaN
7,JCcb65e1,131239638605,Hemoglobin A1c,2017-11-07 23:27:00,6.2,%,None,NaN
8,JCcb65e4,131063755584,Hemoglobin A1c (Manual Entry) See EMR for details,NaT,5.7,% (See scan or EMR data for detail),None,NaN
9,JCcb65ee,14489527,Hemoglobin A1c,2006-05-02 08:38:00,5.7,%,None,NaN


In [46]:
tdd_subqueries_data.to_csv("./Data/a1c.csv")

In [47]:
query.close()

## Diet

Query for Diet

In [24]:
query = open('diet.sql', 'r')

In [25]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [26]:
tdd_subqueries_data.head(10)

,jc_uid,order_proc_id_coded,pat_enc_csn_id_coded,proc_start_time_jittered,description
0,JCcb6601,376862154,131011126327,2010-09-28 12:30:00,DIET NOURISHMENTS
1,JCcb6932,451275286,131045874945,2014-10-18 00:00:00,DIET NPO AFTER MIDNIGHT
2,JCcb6b7c,414240353,131021834984,2012-11-12 18:30:00,BMT DIET
3,JCcb6fa2,584585310,131258217311,2018-11-12 00:00:00,DIET NPO AFTER MIDNIGHT
4,JCcb7169,369520458,131008898162,2010-04-05 17:30:00,DIET NOURISHMENTS
5,JCcb74ce,363665884,131007907827,2010-01-09 17:30:00,DIET CARBOHYDRATE CONTROLLED
6,JCcb79c7,413123750,131020887135,2012-10-31 12:30:00,DIET NUTRITIONAL SUPPLEMENTS
7,JCcb7d8d,373369533,131010248235,2010-08-29 10:30:00,BMT DIET
8,JCcb7d8d,373417625,131010272940,2010-08-30 09:30:00,BMT DIET
9,JCcb7d8d,374116001,131010388819,2010-09-15 07:45:00,BMT DIET


In [27]:
tdd_subqueries_data.to_csv("./Data/diet.csv")

In [28]:
query.close()

In [29]:
tdd_subqueries_data["description"].value_counts()

BMT DIET                                                  2090
DIET NPO AFTER MIDNIGHT                                    451
DIET REGULAR                                               420
DIET LOW MICROBIAL                                         366
DIET NUTRITIONAL SUPPLEMENTS                               311
DIET NOURISHMENTS                                          289
DIET NPO                                                   160
DIET CARBOHYDRATE CONTROLLED                                78
DIET TUBE FEEDING AND MEAL TRAY                             69
DIET CLEAR LIQUID                                           58
DIET NPO AFTER MIDNIGHT FOR SURGERY                         41
DIET REGULAR (SVC)                                          31
DIET LOW SODIUM                                             26
DIET GUEST TRAY                                             21
DIET NPO (SVC)                                              15
DIET CLEAR LIQUID (SVC)                                

In [30]:
print(tdd_subqueries_data["description"].unique())

['DIET NOURISHMENTS' 'DIET NPO AFTER MIDNIGHT' 'BMT DIET'
 'DIET CARBOHYDRATE CONTROLLED' 'DIET NUTRITIONAL SUPPLEMENTS'
 'DIET TUBE FEEDING AND MEAL TRAY' 'DIET REGULAR'
 'DIET POST SURGICAL - STEP TWO' 'DIET COMBINATION' 'DIET REGULAR (SVC)'
 'DIET CARBOHYDRATE CONTROLLED/LOW SODIUM/LOW FAT (SVC)'
 'DIET DYSPHAGIA TEST TRAY' 'DIET GUEST TRAY'
 'DIET CARB CONTROL/LOW SODIUM/LOW CHOL/LOW SAT FAT' 'DIET NPO'
 'DIET CLEAR LIQUID' 'DIET LOW MICROBIAL' 'DIET LOW FIBER LOW RESIDUE'
 'DIET FULL LIQUID' 'DIET NPO AFTER MIDNIGHT FOR SURGERY'
 'DIET LOW SODIUM/LOW FAT (SVC)' 'DIET RENAL' 'DIET SODIUM RESTRICTED'
 'DIET LOW SODIUM LOW CHOLESTEROL LOW SAT FAT' 'DIET POST GASTRIC SURGERY'
 'DIET LOW SODIUM' 'DIET VEGETARIAN' 'DIET TUBE FEEDING ONLY'
 'DIET GLUTEN FREE' 'COMBINATION DIET' 'DIET TUBE FEEDING NO TRAY (SVC)'
 'DIET POST SURGICAL - STEP ONE' 'DIET CLEAR LIQUID (SVC)'
 'RD COMMUNICATION TO DIET OFFICE (SVC)' 'DIET NPO (SVC)'
 'DIET NPO AFTER MIDNIGHT (SVC)'
 'ADVANCE DIET AS TOLERATED (

TUBE FEEDING AND MEAL TRAY; DIET TUBE FEEDING ONLY; 

## Common Labs

Query for Common Labs

In [53]:
query = open('commonLabs.sql', 'r')

In [54]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [55]:
tdd_subqueries_data.head(10)

,lab_name,total
0,"Creatinine, Ser/Plas",224051
1,"Potassium, Ser/Plas",222685
2,"Sodium, Ser/Plas",222329
3,"Chloride, Ser/Plas",221903
4,"CO2, Ser/Plas",221201
5,"Calcium, Ser/Plas",220861
6,"Glucose, Ser/Plas",220252
7,Hematocrit,219534
8,WBC,217221
9,RBC,216993


In [56]:
tdd_subqueries_data.to_csv("./Data/commonLabs.csv")

In [57]:
query.close()

## Creatinine

Query for Creatinine

In [58]:
query = open('creatinine.sql', 'r')

In [59]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [60]:
tdd_subqueries_data.head(10)

,rit_uid,pat_enc_csn_id_coded,lab_name,taken_time_jittered,ord_num_value,reference_unit,result_in_range_yn
0,JCcb657d,6684962,"Creatinine, Ser/Plas",2001-07-08 10:08:00,0.80,mg/dL,Y
1,JCcb657d,6684962,"Creatinine, Ser/Plas",2002-05-04 16:52:00,0.80,mg/dL,Y
2,JCcb657d,6684962,"Creatinine, Ser/Plas",2002-06-10 11:24:00,0.80,mg/dL,Y
3,JCcb657d,6684962,"Creatinine, Ser/Plas",2002-09-21 16:36:00,0.80,mg/dL,Y
4,JCcb657d,6684962,"Creatinine, Ser/Plas",2004-03-28 16:46:00,0.70,mg/dL,None
5,JCcb657d,6684962,"Creatinine, Ser/Plas",2005-05-05 14:32:00,0.80,mg/dL,None
6,JCcb657d,6684962,"Creatinine, Ser/Plas",2006-06-04 13:04:00,0.83,mg/dL,None
7,JCcb657d,6684962,"Creatinine, Ser/Plas",2007-10-15 12:25:00,0.78,mg/dL,None
8,JCcb657d,131009207572,"Creatinine, Ser/Plas",2010-05-24 12:20:00,0.90,mg/dL,None
9,JCcb657d,131079498600,"Creatinine, Ser/Plas",2015-04-10 17:45:00,0.90,mg/dL,None


In [61]:
tdd_subqueries_data.to_csv("./Data/creatinine.csv")

In [62]:
query.close()

## Steroids

Query for Steroids

In [39]:
query = open('steroidQuery.sql', 'r')

In [40]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [41]:
tdd_subqueries_data.head(10)

,jc_uid,order_med_id_coded,sig,dose_unit,route,pat_enc_csn_id_coded,taken_time_jittered,med_description,freq_display_name
0,JCcb6582,454499543,4.0,mg,Intravenous,131060083874,2014-11-19 12:25:00,DEXAMETHASONE SODIUM PHOSPHATE 4 MG/ML INJ SOLN,PRN
1,JCcb65e1,541848146,125.0,mg,Intravenous,131239638605,2017-11-08 07:45:00,METHYLPREDNISOLONE SOD SUC(PF) 125 MG/2 ML INJ...,Once
2,JCcb65e1,541922794,40.0,mg,Feeding Tube,131239638605,2017-11-09 13:43:00,PREDNISONE 20 MG PO TABS,DAILY
3,JCcb65e1,541922794,40.0,mg,Feeding Tube,131239638605,2017-11-10 09:12:00,PREDNISONE 20 MG PO TABS,DAILY
4,JCcb65e1,541922794,40.0,mg,Feeding Tube,131239638605,2017-11-11 08:32:00,PREDNISONE 20 MG PO TABS,DAILY
5,JCcb65e1,541922794,40.0,mg,Feeding Tube,131239638605,2017-11-12 09:09:00,PREDNISONE 20 MG PO TABS,DAILY
6,JCcb65e1,542264105,20.0,mg,Feeding Tube,131239638605,2017-11-13 08:07:00,PREDNISONE 20 MG PO TABS,DAILY
7,JCcb65e1,542453808,20.0,mg,Oral,131239638605,2017-11-14 09:16:00,PREDNISONE 20 MG PO TABS,DAILY
8,JCcb65e1,542453808,20.0,mg,Oral,131239638605,2017-11-15 08:18:00,PREDNISONE 20 MG PO TABS,DAILY
9,JCcb65e1,542453808,20.0,mg,Oral,131239638605,2017-11-16 12:25:00,PREDNISONE 20 MG PO TABS,DAILY


In [42]:
tdd_subqueries_data.to_csv("./Data/steroids.csv")

In [43]:
query.close()

## Weight

Query for Weight

In [30]:
query = open('weight.sql', 'r')

In [ ]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [ ]:
tdd_subqueries_data.head(10)

In [ ]:
tdd_subqueries_data.to_csv("./Data/weight.csv")

In [ ]:
query.close()

## Height

Query for Height

In [35]:
query = open('height.sql', 'r')

In [ ]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [ ]:
tdd_subqueries_data.head(10)

In [ ]:
tdd_subqueries_data.to_csv("./Data/height.csv")

In [ ]:
query.close()

## Glucoses

Query for Glucose values (first value in admission)

In [44]:
query = open('glucoses.sql', 'r')

In [45]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [46]:
tdd_subqueries_data.head(10)

,rit_uid,pat_enc_csn_id_coded,lab_name,ord_value,ord_num_value,taken_time_jittered,result_time_jittered
0,JCd59f64,131248252494,"Glucose, Ser/Plas",123,123.0,2018-03-20 13:30:00,2018-03-20 14:36:00
1,JCd4df71,131240996220,"Glucose, Ser/Plas",150,150.0,2017-12-26 07:49:00,2017-12-26 08:11:00
2,JCe4e1e8,131178145935,"Glucose, Ser/Plas",187,187.0,2016-02-07 04:45:00,2016-02-07 05:17:00
3,JCdc0cfa,131240421000,"Glucose, Ser/Plas",131,131.0,2017-11-25 02:46:00,2017-11-25 03:10:00
4,JCd66cc3,131237394594,"Glucose, Ser/Plas",126,126.0,2017-07-23 16:31:00,2017-07-23 17:36:00
5,JCd20376,131234429925,"Glucose, Ser/Plas",115,115.0,2017-06-24 14:54:00,2017-06-24 15:39:00
6,JCdb84b4,131228966152,"Glucose, Ser/Plas",183,183.0,2017-05-09 05:40:00,2017-05-09 06:26:00
7,JCe003bf,131239633350,"Glucose, Ser/Plas",84,84.0,2017-11-10 18:53:00,2017-11-10 19:33:00
8,JCeb4cb9,131193965421,"Glucose, Ser/Plas",97,97.0,2016-07-30 09:29:00,2016-07-30 10:09:00
9,JCccc72a,131202703847,"Glucose, Ser/Plas",110,110.0,2016-09-26 11:18:00,2016-09-26 13:04:00


In [47]:
tdd_subqueries_data.to_csv("./Data/glucoses.csv")

In [48]:
query.close()

Larry's Queries to Write:
- Weight, BMI, Admission BG
- Lab Results: CBC, ESR (erthrocyte), CRP (c reactive protein), CHEM 10, Liver function test
- Medications (steroid or not)
- # of diagnoses
- Procedures in the last 48 hours

## TPN/PPN

Query for TPN/PPN people

In [19]:
query = open('TPN_PPN.sql', 'r')

In [20]:
tdd_subqueries_data = pd.read_sql_query(query.read(), conn)

In [21]:
tdd_subqueries_data.head(10)

,jc_uid,pat_enc_csn_id_coded,med_description,medication_id,order_time_jittered,start_time_jittered
0,JCcc0528,16624126,TPN ADULT STANDARD,550000,2008-10-10 19:10:00,NaT
1,JCce2fbc,131007070249,TPN ADULT STANDARD,550000,2009-10-15 14:43:00,NaT
2,JCd4b533,131181914407,TPN ADULT CYCLIC,3232,2016-03-19 16:40:00,2016-03-19 21:00:00
3,JCce469c,16426096,TPN BMT,27617,2008-09-03 11:04:00,2008-09-03 20:00:00
4,JCcba0a4,131010279813,TPN ADULT STANDARD,27617,2010-09-17 10:56:00,2010-09-17 20:00:00
5,JCcd3a24,131014406974,TPN ADULT STANDARD,3232,2011-09-21 11:05:00,2011-09-21 20:00:00
6,JCd7105d,131041353367,TPN ADULT CYCLIC,27617,2014-09-22 11:33:00,2014-09-22 21:00:00
7,JCdb5074,16875291,TPN ADULT STANDARD,3232,2008-10-11 08:08:00,2008-10-11 20:00:00
8,JCdf68e7,131024625892,TPN ADULT STANDARD,27617,2013-07-07 11:49:00,2013-07-07 20:00:00
9,JCe25c8e,131030378700,TPN ADULT STANDARD,27617,2014-04-22 14:17:00,2014-04-22 21:00:00


In [22]:
tdd_subqueries_data.to_csv("./Data/tpn_ppn.csv")

In [23]:
query.close()